In [3]:
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('data/reviews.csv')

In [5]:
data = data[data['review'].apply(lambda x: isinstance(x, str))]
data = data[data['stars'].apply(lambda x: not np.isnan(x))]
data['created_at'] = data['created_at'].apply(pd.to_datetime)
data['year'] = data['created_at'].apply(lambda x: x.year)
train_sample = data[data['year'] < 2018]
test_sample = data[data['year'] == 2018]
data.head(5)

,review_id,review,created_at,product_id,product_name,stars,year
1,52414,就算保质期一个月，还有一个星期过期的巧克力也拿到平台上卖？,2017-01-13,96514296,日本北海道进口royce生巧克力圣诞节情人节礼物巧克力礼盒装 纯黑苦【现货当天发】,1.0,2017
2,40670,太贵！其他的都可以！,2016-09-25,79848693,【哈肉联】风干香肠 单根50g 瘦肉块有嚼劲 哈尔滨特产齐名红肠 *24根 [整箱],4.0,2016
3,12727,很理想 满意,2016-12-25,79295853,天之缘 一次性双喜红地毯结婚地毯开业庆典门垫地毯婚庆红地毯 百年好合双喜红地毯（10米价格）,5.0,2016
4,9456,满满的狗粮！谁要吃,2017-06-04,74142329,比瑞吉天然粮 狗粮 比熊室内成犬粮2kg,5.0,2017
6,75359,只要老婆喜欢就好,2017-12-16,94059728,徐福记 什锦糖果散装水果糖软硬糖结婚喜糖批发 多口味夹心棉花糖500g,3.0,2017


In [80]:
data['review']
symbols = []
for review in data['review']:
    symbols += list(review)
    
grouped_dict = {}    
for symbol in symbols:
    if symbol in grouped_dict:
        grouped_dict[symbol] += 1
    else: 
        grouped_dict[symbol] = 1
        
sorted_words = sorted([(k, v) for k, v in grouped_dict.items()], key=lambda x: x[1], reverse=True)
word2index = {}
index2word = {}
index2count = []
for idx, val in enumerate(sorted_words):
    index2word[idx] = val[0]
    word2index[val[0]] = idx
    index2count.append((idx, val[1]))
    
data['review_indexes'] = data['review'].apply(lambda x: list(map(lambda word: word2index[word], list(x))))

In [67]:
x_train = data[data['year'] < 2018]['review_indexes']
y_train = data[data['year'] < 2018]['stars']
x_test = data[data['year'] == 2018]['review_indexes']
y_test = data[data['year'] == 2018]['stars']

In [2]:
# the histogram of the data
n, bins, patches = plt.hist(map(lambda x: x[0], index2count), 100)
plt.grid(True)

NameError: name 'plt' is not defined

In [117]:
map(lambda x: x[0], index2count)

[(0, 192389),
 (1, 83159),
 (2, 69651),
 (3, 59308),
 (4, 57855),
 (5, 54253),
 (6, 47364),
 (7, 40809),
 (8, 40133),
 (9, 37282),
 (10, 32239),
 (11, 31444),
 (12, 31130),
 (13, 30039),
 (14, 25294),
 (15, 22207),
 (16, 20893),
 (17, 20393),
 (18, 20151),
 (19, 19117),
 (20, 18895),
 (21, 18398),
 (22, 16654),
 (23, 15492),
 (24, 15315),
 (25, 15085),
 (26, 15001),
 (27, 14986),
 (28, 14937),
 (29, 13894),
 (30, 13888),
 (31, 13600),
 (32, 13421),
 (33, 13339),
 (34, 13282),
 (35, 13217),
 (36, 11637),
 (37, 11409),
 (38, 11231),
 (39, 11193),
 (40, 10960),
 (41, 10833),
 (42, 10831),
 (43, 10743),
 (44, 10729),
 (45, 10706),
 (46, 10541),
 (47, 10470),
 (48, 10444),
 (49, 10296),
 (50, 10296),
 (51, 10242),
 (52, 10073),
 (53, 9991),
 (54, 9929),
 (55, 9847),
 (56, 9656),
 (57, 9637),
 (58, 9544),
 (59, 9498),
 (60, 9371),
 (61, 9332),
 (62, 9166),
 (63, 9128),
 (64, 9117),
 (65, 9115),
 (66, 8976),
 (67, 8884),
 (68, 8822),
 (69, 8711),
 (70, 8648),
 (71, 8252),
 (72, 8207),
 (73, 8

In [5]:
model = Word2Vec(data['review'], size=300, window=10, workers=4)

In [35]:
unique_symbols = set()
# data['review'].apply(lambda x: set(x))
for review in data['review']:
    unique_symbols = unique_symbols | set(review)

In [49]:
sorted(list(unique_symbols), key=str.lower)

['\x01',
 '\x08',
 '\x14',
 ' ',
 '!',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'A',
 'b',
 'B',
 'C',
 'c',
 'd',
 'D',
 'E',
 'e',
 'f',
 'F',
 'g',
 'G',
 'h',
 'H',
 'I',
 'i',
 'J',
 'j',
 'K',
 'k',
 'L',
 'l',
 'M',
 'm',
 'N',
 'n',
 'o',
 'O',
 'P',
 'p',
 'q',
 'Q',
 'r',
 'R',
 'S',
 's',
 't',
 'T',
 'U',
 'u',
 'V',
 'v',
 'W',
 'w',
 'X',
 'x',
 'Y',
 'y',
 'z',
 'Z',
 '{',
 '|',
 '}',
 '~',
 '·',
 'ā',
 'Ő',
 'ǒ',
 'ʔ',
 'ʕ',
 'ˇ',
 '˘',
 '˙',
 '˵',
 '̀',
 '́',
 '̊',
 '̥',
 '̫',
 '̳',
 '̴',
 '͒',
 '͓',
 '͜',
 '͞',
 '͟',
 '͡',
 'ϵ',
 '϶',
 'а',
 'б',
 'в',
 'г',
 'д',
 'Д',
 'е',
 'ж',
 'з',
 'и',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'х',
 'ч',
 'ш',
 'щ',
 'ы',
 'ь',
 'ю',
 'Я',
 'я',
 '҂',
 'ԅ',
 'Ծ',
 '\u0602',
 'و',
 '٩',
 'ڡ',
 'ۖ',
 'ۣ',
 '۶',
 'ू',
 '૭',
 '୧',
 'இ',
 '౨',
 

In [55]:
data[data['product_name'].str.contains('Dove')]

,review_id,review,created_at,product_id,product_name,stars,year
7,75107,撇撇撇 太撇了,2017-05-07,89061649,德芙（Dove） 德芙巧克力礼盒装结婚婚庆喜糖糖果送女朋友生日礼品商务回礼平安夜圣诞糖果 红...,1.0,2017
25,9712,宝贝收到了，真心不错,2017-06-13,79294457,德芙（Dove） 巧克力棒巧丝60支装威化巧克力棒保温包装休闲零食糖果送女友,5.0,2017
137,60556,商品很不错，我挺喜欢的,2018-02-04,79295183,德芙（Dove） 纯黑巧克力252g*3碗装66%醇黑苦巧克力结婚喜糖休闲零食聚会糖果,5.0,2018
161,45372,包装完整，是女儿超喜欢超喜欢的口味，看起来还不错哦，就是物流稍稍有点慢。,2018-01-27,79293541,德芙（Dove） 巧克力分享碗装 糖果巧克力休闲零食丝滑牛奶香浓黑等多种口味可选情人节年货 ...,5.0,2018
174,96218,很不错的商品，非常好,2017-10-01,81215134,德芙（Dove） 巧克力创意礼盒装情人节礼物送女朋友老婆生日礼品礼物零食新年糖果 爱丽莎红色花礼盒,5.0,2017
204,10750,退款啊 朋友,2018-02-03,107441097,德芙（Dove） 榛仁葡萄干巧克力盒装（14gx16块）元旦节日礼物休闲零食品 浓情时光22...,1.0,2018
221,13328,好精致，速度快，价格不怎么美丽，慎重',2017-11-25,79877508,德芙（Dove） 爱心铁盒德芙巧克力礼盒装丝滑牛奶4.5g结婚婚庆喜糖成品含糖果商务礼品 天蓝色,2.0,2017
310,87606,不错，，不错啊，，，,2016-09-23,90206148,Dove 德芙巧克力多口味礼盒装礼物送闺蜜送女友520礼物包邮 奶香白口味,NaN,2016
351,55128,嗯嗯嗯 给妈妈买的，快递有点慢，生日过后妈妈才收到。,2017-12-03,99234289,德芙（Dove） 巧克力分享碗装 榛仁葡萄干巧克力糖果巧克力休闲零食243g情人节年货生日,2.0,2017
353,44889,。。。。。。。。。。。。。。。。。。,2016-04-09,79295793,德芙（Dove） 德芙士力架花生夹心巧克力约20g/个旅游熬夜能量棒婚庆喜糖果凑单商品,5.0,2016
